# Nonlinear Analysis Notebook
### Importing libraries

In [ ]:
import numpy as np 
import h5py
import matplotlib.pyplot as plt
import scipy.io
import os
import re
from utils import *

### Loading runs

In [ ]:
###############################################
runNumbers = []  # enter the run numbers to be loaded
folder = '/sdf/data/lcls/ds/cxi/cxil1037623/hdf5/smalldata/' # insert here the directory path where .h5 files are stored
###############################################
# (1) keys_to_combine: some keys loaded for each shot & stored per shot 
# (2) keys_to_sum: some keys loaded per each run and added 
# (3) keys_to_check : check if some keys exits and have same values in all runs and load these keys 
keys_to_combine = ['jungfrau4M/azav_azav',
                   'ipm_dg2/sum',
                    #'ipm_dg3/sum',
                   #'ipm_hfx_dg2/sum',
                   #'alvium_u1/laser_com',
                   #'alvium_u1/laser_sum',
                   #'Acqiris/FastTiming_area',
                   'gas_detector/f_11_ENRC',
                   'ebeam/photon_energy',
                   'evr/code_183',
                   'evr/code_137',
                   'evr/code_141',
                   'lightStatus/laser',
                   'lightStatus/xray']

keys_to_sum = ['Sums/jungfrau4M_calib']
#               'Sums/jungfrau4M_calib_thresADU1']

keys_to_check = ['UserDataCfg/jungfrau4M/azav__azav_q',
                'UserDataCfg/jungfrau4M/azav__azav_qbin',
                'UserDataCfg/jungfrau4M/azav__azav_qbins',
                 'UserDataCfg/jungfrau4M/x',
                 'UserDataCfg/jungfrau4M/y',
                 'UserDataCfg/jungfrau4M/z',
                 'UserDataCfg/jungfrau4M/azav__azav_matrix_q',
                 'UserDataCfg/jungfrau4M/azav__azav_matrix_phi',
                #'UserDataCfg/jungfrau4M/cmask',
                # 'UserDataCfg/jungfrau4M/Full_thres__Full_thres_thresADU',
                # 'UserDataCfg/jungfrau4M/Full_thres__Full_thres_bound',
                'UserDataCfg/jungfrau4M/common_mode_pars']
# Load the data in
data = combineRuns(runNumbers, folder, keys_to_combine, keys_to_sum, keys_to_check, verbose=False)  # this is the function to load the data with defined keys

azav = np.squeeze(data['jungfrau4M/azav_azav']) # I(q) : 1D azimuthal average of signals in each q bin
qbin = data['UserDataCfg/jungfrau4M/azav__azav_qbin'] # q bin-size
q = data['UserDataCfg/jungfrau4M/azav__azav_q'] # q bins 
qbins = data['UserDataCfg/jungfrau4M/azav__azav_qbins'] # q bins
matrix_q = data['UserDataCfg/jungfrau4M/azav__azav_matrix_q'].reshape(8,512,1024)  # q matrix Jungfrau shaped
matrix_phi = data['UserDataCfg/jungfrau4M/azav__azav_matrix_phi'].reshape(8,512,1024) # phi for anisotropic data
laserOn = data['evr/code_183'].astype(bool)  # laser on events
xrayOn = data['evr/code_137'].astype(bool)  # xray on events
laserOn2 = data['lightStatus/laser'].astype(bool)  # laser on events 
xrayOn2 = data['lightStatus/xray'].astype(bool)  # xray on events
jungfrau_sum = data['Sums/jungfrau4M_calib']  # Total Jungfrau detector counts summed in a run
#jungfrau_sum = data['Sums/jungfrau4M_calib_thresADU1']   # Total Jungfrau detector counts with Thresholds added, summed in a run 
x = data['UserDataCfg/jungfrau4M/x'] # coordinates of Jungfrau detector x,y,z
y = data['UserDataCfg/jungfrau4M/y']
z = data['UserDataCfg/jungfrau4M/z'] 

#cmask = data['UserDataCfg/jungfrau4M/cmask'].astype(bool) # Mask for detector created 
run_indicator = data['run_indicator'] # run indicator for each shot
# UV_com = data['alvium_u1/laser_com']  # center of mass for laser (pump) pointing
# UV_sum = data['alvium_u1/laser_sum']   # intensity sum for laser (pump) pointing
# Acqiris_FastTiming = data['Acqiris/FastTiming_area']  # diode signal for laser after interaction
# Acqiris_Laser = data['Acqiris/Laser_area'] # diode signal for laser before interaction
dg2 = data['ipm_dg2/sum']   # upstream diode x-ray intensity
# dg3 = data['ipm_dg3/sum']    # downstream diode x-ray intensity
# pressure = data['epicsAll/gasCell_pressure']  # pressure in gas cell
xray_energy = data['gas_detector/f_11_ENRC']   # xray energy from gas detector (not calibrated to actual values)
xray_eV = data['ebeam/photon_energy']    # x-ray energy energy in eV